In [25]:
# -*- coding: utf-8 -*-
# -*- authors : Vincent Roduit, Filippo Quadri -*-
# -*- date : 2024-05-03 -*-
# -*- Last revision: 2024-05-03 -*-
# -*- python version : 3.9.18 -*-
# -*- Description: Notebook that summarize results-*-

# <center> EE - 451 Image Analysis and Pattern recognition </center>
## <center> Ecole Polytechnique Fédérale de Lausanne </center>
### <center>Coin Challenge </center>
---

In [184]:
#Import libraries
import os
import numpy as np
import torch
import importlib
import skimage as sk
from skimage.color import rgb2hsv
import matplotlib.pyplot as plt
from typing import Callable
import cv2 as cv
from skimage.morphology import closing, opening, disk, remove_small_holes, remove_small_objects
from ipywidgets import interact, interactive, fixed, interact_manual
import pandas as pd
from torch.utils.data import Dataset, DataLoader

import warnings
warnings.filterwarnings("ignore")

In [190]:
#Import files
from data_classes.ref_data import refCoin
from data_classes.train_data import trainCoin
from data_classes.test_data import testCoin
import constants
from visualization import *
importlib.reload(constants)
from processing.process_func import *
from models.utils import *
from processing.data_augmentation import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. Load different Datasets

In [4]:
ref_data = refCoin()

Loading data from pickle files


In [5]:
train_data = trainCoin(save=False)

Loading data from pickle files


In [6]:
test_data = testCoin()

Loading data from pickle files


# 2 Create Data Ready for Neural Network

In [11]:
train_data.process_images()
train_data.create_masked_images()

In [12]:
train_data.create_coin_images()

In [14]:
conversion_table = get_classes_conv_table()

In [34]:
coin_labels = get_coin_labels()

In [98]:
images, labels, df_images_labels = create_data_structure(train_data.coins, coin_labels, conversion_table)

In [105]:
train_images, train_labels, val_images, val_labels = create_datasets(images, labels)

# 3 Data Augmentation

In [167]:
train_images_aug, train_labels_aug = augment_set(train_images, train_labels)

In [191]:
train_images_aug, train_labels_aug = augment_blur(train_images_aug, train_labels_aug)   

In [199]:
val_images_aug, val_labels_aug = augment_set(val_images, val_labels)

# 4 Train Neural Network


In [206]:
train_dataloader, val_dataloader = create_dataloader(train_images, train_labels, val_images, val_labels)

In [101]:
from models.cnn import Basic_CNN
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch import nn

image_dim = images.shape[1]
num_classes = len(conversion_table)
# Define the model
cnn = Basic_CNN(image_size=image_dim, num_classes=num_classes) 

# Define the optimizer
optimizer = torch.optim.Adam(cnn.parameters(), lr=0.001)
# Define the scheduler
scheduler = ReduceLROnPlateau(
    optimizer, mode="min", factor=0.1, patience=2, verbose=True
)
criterion = nn.BCEWithLogitsLoss()

# Train the model
cnn.train_model(
    optimizer,
    scheduler,
    train_dataloader,
    val_dataloader,
)

Epoch 1/10, Loss: 3.0634966323773067, Validation Accuracy: 0.4000
Epoch 2/10, Loss: 2.406124475846688, Validation Accuracy: 0.4000


KeyboardInterrupt: 